# EDA_Statistical_Thinking

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set visual styles for plots
sns.set_style('whitegrid')
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Robustly load the provided dataset (current file: data/MachineLearningRating_v3.txt)
file_path = 'data/MachineLearningRating_v3.txt'
try:
    # let pandas sniff the separator if possible
    data = pd.read_csv(file_path, sep=None, engine='python', low_memory=False)
    print(f"Data loaded successfully from {file_path}.")
except Exception as e:
    print(f"Error loading {file_path}: {e}")
    # try an alternative relative path
    alt = './data/MachineLearningRating_v3.txt'
    try:
        data = pd.read_csv(alt, sep=None, engine='python', low_memory=False)
        print(f"Data loaded successfully from {alt}.")
    except Exception as e2:
        raise FileNotFoundError(f"Could not load dataset from {file_path} or {alt}: {e2}")

print("\n--- Initial Data Snapshot ---")
print(data.head())

print("\n--- Columns detected ---")
print(list(data.columns))

## Section 2: Data Structure and Quality Assessment (Task 1.2)

### 2.1 Data Structure (KPI: Data Structure)
print("\n--- Data Information (Data Types and Non-Null Counts) ---")
print(data.info())

# Detect a date-like column and coerce to datetime into 'TransactionDate'
date_cols = [c for c in data.columns if ('date' in c.lower()) or ('transaction' in c.lower()) or ('month' in c.lower())]
if date_cols:
    date_col = date_cols[0]
    data['TransactionDate'] = pd.to_datetime(data[date_col], errors='coerce')
    print(f"\nParsed date-like column '{date_col}' into 'TransactionDate' (nulls if parsing failed).")
else:
    print("\nNo date-like column found; 'TransactionDate' not created.")

### 2.2 Data Quality Assessment (KPI: Missing Values)
print("\n--- Missing Value Check ---")
missing_data = data.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
print(missing_data)

### 2.3 Descriptive Statistics (KPI: Variability)
print("\n--- Descriptive Statistics for Financial/Numerical Features ---")
# Focus on common financial and numerical columns (pick those that exist)
possible_numerical = ['TotalPremium', 'TotalClaims', 'SumInsured', 'CustomValueEstimate', 'Cubiccapacity', 'Kilowatts']
numerical_cols = [c for c in possible_numerical if c in data.columns]
if numerical_cols:
    print(data[numerical_cols].describe())
else:
    print("None of the expected numerical columns were found: " + str(possible_numerical))

# Variability check for TotalPremium and TotalClaims if present
if 'TotalPremium' in data.columns and 'TotalClaims' in data.columns:
    premium_std = data['TotalPremium'].std()
    claims_std = data['TotalClaims'].std()
    print(f"\nVariability (Standard Deviation):")
    print(f"TotalPremium: {premium_std:,.2f}")
    print(f"TotalClaims: {claims_std:,.2f}")
else:
    print("Cannot compute std for 'TotalPremium'/'TotalClaims' — one or both columns missing.")


## Section 3: Exploratory Data Analysis (EDA) - Initial Insights

### 3.1 Overall Loss Ratio Calculation
# Loss Ratio = TotalClaims / TotalPremium (summed over the portfolio)
if ('TotalPremium' in data.columns) and ('TotalClaims' in data.columns):
    total_premium = data['TotalPremium'].sum(skipna=True)
    total_claims = data['TotalClaims'].sum(skipna=True)
    if total_premium > 0:
        overall_loss_ratio = total_claims / total_premium
        print(f"\n--- Overall Portfolio Loss Ratio ---")
        print(f"Overall Loss Ratio: {overall_loss_ratio:.4f} (or {overall_loss_ratio*100:.2f}%)")
    else:
        print("TotalPremium sums to zero; cannot compute overall loss ratio.")
else:
    print("Columns 'TotalPremium' and/or 'TotalClaims' missing; skipping loss ratio calculation.")

# --- Continue your EDA here with the required Univariate, Bivariate, and Multivariate analysis ---
# E.g., Calculating Loss Ratio by Province, plotting histograms, etc.

# You can start calculating loss ratio by Province here if columns exist:
if ('Province' in data.columns) and ('TotalPremium' in data.columns) and ('TotalClaims' in data.columns):
    print("\n--- Loss Ratio by Province ---")
    province_risk = data.groupby('Province').agg("
    TotalPremium=('TotalPremium', 'sum'),",
,
,
,
,
,

# Create 'HadClaim' flag from 'TotalClaims' if not present
if 'HadClaim' not in data.columns:
    if 'TotalClaims' in data.columns:
        data['HadClaim'] = (data['TotalClaims'] > 0).astype(int)
        print("Created 'HadClaim' from 'TotalClaims'.")
    else:
        data['HadClaim'] = np.nan
        print("'HadClaim' not created — 'TotalClaims' missing.")

# You would then visualize this as one of your required plots.

#

: 

# Temporal Trends Analysis

In [ ]:

# Aggregate data by month
monthly_data = data.groupby(pd.Grouper(key='TransactionDate', freq='M')).agg(
    TotalMonthlyClaims=('TotalClaims', 'sum'),
    PolicyCount=('PolicyID', 'count'),
    ClaimCount=('HadClaim', 'sum')
).reset_index()

monthly_data['ClaimFrequency'] = monthly_data['ClaimCount'] / monthly_data['PolicyCount']
# Calculate Claim Severity: Total Claims / Number of Claims
monthly_data['ClaimSeverity'] = monthly_data['TotalMonthlyClaims'] / monthly_data['ClaimCount']

# Plotting the trend (Plot 1 of 3: Creative Plot)
plt.figure(figsize=(14, 6))
sns.lineplot(x='TransactionDate', y='ClaimSeverity', data=monthly_data, label='Claim Severity (Rand)')
sns.lineplot(x='TransactionDate', y='ClaimFrequency', data=monthly_data, label='Claim Frequency (Proportion)', color='red')
plt.title('Temporal Trend: Claim Severity and Claim Frequency (Feb 2014 - Aug 2015)')
plt.ylabel('Value')
plt.xlabel('Transaction Month')
plt.legend()
plt.show()
#

# Outlier Detection (Box Plots)

In [ ]:

# Set up figure for box plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Box Plot 1: TotalClaims (Zoomed in for clarity)
sns.boxplot(y=data['TotalClaims'], ax=axes[0])
axes[0].set_title('Box Plot of TotalClaims (Zoomed)')
# Limit y-axis to focus on the bulk of the data and show extreme outliers as dots
axes[0].set_ylim(0, data['TotalClaims'].quantile(0.99)) 
axes[0].set_ylabel('Total Claims Amount (Rand)')

# Box Plot 2: CustomValueEstimate (Zoomed in for clarity)
sns.boxplot(y=data['CustomValueEstimate'], ax=axes[1])
axes[1].set_title('Box Plot of CustomValueEstimate (Zoomed)')
axes[1].set_ylim(0, data['CustomValueEstimate'].quantile(0.99))
axes[1].set_ylabel('Custom Value Estimate (Rand)')

plt.tight_layout()
plt.show()

# Creative Plot 2: Geographical Risk Comparison (Loss Ratio)

In [ ]:


# Filter for provinces with substantial volume (e.g., more than 1000 policies)
province_volume = data['Province'].value_counts()
provinces_to_plot = province_volume[province_volume >= 1000].index

province_risk_plot = calculate_loss_ratio(data[data['Province'].isin(provinces_to_plot)], 'Province')
province_risk_plot = province_risk_plot.sort_values(by='LossRatio', ascending=True)

plt.figure(figsize=(12, 7))
sns.barplot(x=province_risk_plot.index, y=province_risk_plot['LossRatio'] * 100, palette='viridis')
plt.title('Loss Ratio (%) by Province (Actionable Insight)')
plt.xlabel('Province')
plt.ylabel('Loss Ratio (%)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
#

# Creative Plot 3: Risk by Vehicle Make

In [ ]:

top_makes = data['Make'].value_counts().head(10).index.tolist()
make_risk_plot = data[data['Make'].isin(top_makes)].copy()

# Calculate Claim Severity for the top makes
make_severity = make_risk_plot[make_risk_plot['HadClaim'] == 1].groupby('Make')['TotalClaims'].mean().reset_index()
make_severity = make_severity.sort_values(by='TotalClaims', ascending=False)

plt.figure(figsize=(12, 7))
sns.barplot(x='Make', y='TotalClaims', data=make_severity, palette='inferno')
plt.title('Average Claim Severity (Rand) for Top 10 Vehicle Makes')
plt.xlabel('Vehicle Make')
plt.ylabel('Average Claim Amount (Rand)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
#